<a href="https://colab.research.google.com/github/Nada-Nasser/images-classification/blob/main/image_classification_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Download All Needed packages**

*   colabcode
*   torchvision
*   nlkt
*   fastapi





In [ ]:
pip install colabcode

In [ ]:
pip install torchvision

In [ ]:
pip install nltk

## **Download image classes dataset**

In [ ]:
!wget https://raw.githubusercontent.com/Lasagne/Recipes/master/examples/resnet50/imagenet_classes.txt

In [ ]:
import nltk
nltk.download('wordnet')

This code loads the images datasat in a list

In [ ]:
with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]

In [ ]:
pip install fastapi

import all packages needed in generating public url

In [ ]:
from colabcode import ColabCode
from fastapi import FastAPI
import uvicorn

In [ ]:
cc=ColabCode(port=12000,code=False)

## Receive Post request to classify the image included in its body

Using **`fastapi`** lib, we create a path to be used in a post request url to call the function named **`classifyimage`**, in the post request body, there should be a json object that has an image in 64-base format.
* request body ex.:
```
{
  file:"64-base encoded image"
}
```

in **`classifyimage`** function:

1.   Convert the 64-base encoded image to **`png`** file and save the generated image in "**`imageToSave.png`**" file.
2.   call **`get_animal_name`** to get the animal name from the image using image recognition model.
3. replace all white spaces in the animal name with underscore ( **`_`** ).
4. convert all letters in the animal name to lowercase.
5. use **`Classify`** class to classify the animal using its name and return **`True`** if it not a wild animal and return **`False`** otherwise






In [ ]:
import base64
from pydantic import BaseModel
from PIL import Image

class Data(BaseModel):
    file: str

app=FastAPI()

@app.post("/classifyimage/")
def classifyimage(image64base : Data):
  print("in classify")
  if(image64base.file == ""): 
    return ["False"]
  
  with open("imageToSave.png", "wb") as fh:
    fh.write(base64.b64decode(image64base.file))
  img = Image.open("imageToSave.png")

  animal_name = get_animal_name(img)
  animal_name = animal_name.replace(" ", "_")
  animal_name = animal_name.lower()
  print(animal_name)
  names = animal_name.split(",_")
  t = Calssify()
  return t.classify(names[0])

## **Calssify class**
Using wordnet synsets and hypernyms, this class used to know if the animal_name is belong to **`domestic_animal`**, **`birds`** or **`entity`** using **dfs** algorithm.
Then if the animal_name belongs to  **`domestic_animal`**, **`birds`** then the function returns **`True`**, otherwise returns **`False`**

In [ ]:
from nltk.corpus import wordnet as wn

pet = wn.synset('domestic_animal.n.01')
#pet = wn.synset('animal.n.01')
entity = wn.synset('entity.n.01')
bird = wn.synset('bird.n.01')

class Animal:
    PET  = 1
    BIRD = 2
    NONE = 3

class Calssify:
    def __init__(self):
        self.found = Animal.NONE
         
    def _DFSUtil(self,node):
        if(node == entity):
            self.found = Animal.NONE
            return
        if(node == bird):
            self.found = Animal.BIRD
            return
        if(node == pet):
            self.found = Animal.PET
            return               
        for neighbour in node.hypernyms():
            if node != entity:
                self._DFSUtil(neighbour)  

    def _DFS(self,node):
        self._DFSUtil(node)
        return self._printFound()
        
    def _printFound(self):
        if self.found == Animal.PET:
            print("IT IS A Pet")
            return ["True"]
        if self.found == Animal.BIRD:
            print("IT IS A BIRD")
            return ["True"]
        if self.found == Animal.NONE:
            print("NOT Domestic Animal")
            return ["False"]
    
    def classify(self,name):
        self.found = Animal.NONE
        if wn.synsets(name):
            print(name)
            name = name+".n.01"
            node = wn.synset(name)
            return self._DFS(node)
        else:
            print("NOT Animal")
            return ["False"]


In [ ]:
from torchvision import models
import torch

alexnet = models.alexnet(pretrained=True)
print(alexnet)

from torchvision import transforms
transform = transforms.Compose([            #[1]
  #transforms.ToPILImage(),
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 #transforms.RandomHorizontalFlip(), 
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

alexnet.eval()

In [ ]:
def get_animal_name(img):
  img_t = transform(img)
  batch_t = torch.unsqueeze(img_t, 0)
  out = alexnet(batch_t)
  print(out.shape)
  
  _, index = torch.max(out, 1)
  #percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
  #print(classes[index[0]], percentage[index[0]].item())
  print(classes[index[0]]) 
  animal_name = classes[index[0]] # we need to convert all letters to lower case and replace spaces with underscore
  return animal_name

In [ ]:
cc.run_app(app=app)